In [1]:
import pandas as pd
X_new = pd.read_csv('C:/Users/RH/A_whole_try_prog/AAC/AAC_embedding.csv', delimiter=',',header = None,index_col= 0, skiprows=1)
X_out = pd.read_csv('C:/Users/RH/A_whole_try_prog/AAC/X_test_AAC.csv',delimiter=',',header = None,index_col= 0, skiprows=1)
X_out_Sequence = pd.read_csv('C:/Users/RH/A_whole_try_prog/X_test_nr.csv',delimiter=',',header = None,index_col= 0, dtype={0: int},skiprows=1)
# load numpy array from csv file
from numpy import loadtxt
import numpy as np
# load array
y_new = loadtxt('C:/Users/RH/A_whole_try_prog/AAC/y_clean_AAC.csv', delimiter=',')


In [2]:
print(X_new.shape)
print(y_new.shape)
print(np.count_nonzero(y_new==0))
print(np.count_nonzero(y_new==1))

(877, 20)
(877,)
310
567


In [3]:
import statistics
from sklearn.metrics import balanced_accuracy_score
from self_function import evaluation as eva

In [4]:
# dataset splitting 
from sklearn.model_selection import train_test_split
X_train_whole, X_ind_test, y_train_whole, y_ind_test =  train_test_split(X_new, y_new, test_size=0.2, random_state=2001)

In [5]:
# logistic regresion l2 penality and class_weight
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
clf = LogisticRegression(class_weight='balanced',penalty= 'l2', max_iter = 5000)
param_grid = {'C':[8,10,20,40,100,200],'solver':['lbfgs','saga']}
grid_search = GridSearchCV(clf,param_grid,cv=10,n_jobs=-1,scoring='precision')
grid_search.fit(X_train_whole,y_train_whole)
best_model_reg = grid_search.best_estimator_
y_pred = best_model_reg.predict(X_out)



#print the model's parameters and validation score
print("About model development")
print("Best Parameters:{}".format(grid_search.best_params_))
print("Best cross_validation socre:{:.3f}".format(grid_search.best_score_))
print("Test set score:{:.2f}".format(grid_search.score(X_ind_test,y_ind_test)))
print("Best_estimator:\n{}".format(grid_search.best_estimator_))

#model performance evaluation:BACC & recall & accuracy & MCC & f1 score& roc_auc
evaluation = eva(best_model_reg,X_new,y_new)
print(evaluation)



# get the probability of class 1
y_pred_prob = best_model_reg.predict_proba(X_out)[:, 1]

# Sort the predictions according to the probability of class 1
sorted_indices = np.argsort(y_pred_prob)[::-1]  # Returns the index in descending order

# Getting predictions and probabilities by sorted index
sorted_y_pred = y_pred[sorted_indices]
sorted_prob = y_pred_prob[sorted_indices]

sorted_sequence = X_out_Sequence.iloc[sorted_indices]

output = np.column_stack((sorted_sequence, sorted_y_pred, sorted_prob))

print("Sorted Sequence, Predictions, and Probabilities:")
print(output)

About model development
Best Parameters:{'C': 8, 'solver': 'lbfgs'}
Best cross_validation socre:0.968
Test set score:0.97
Best_estimator:
LogisticRegression(C=8, class_weight='balanced', max_iter=5000)
BACC =  0.901 ± 0.035
Recall =  0.839 ± 0.054
Precision =  0.939 ± 0.035
MCC =  0.817 ± 0.067
F1 score =  0.886 ± 0.043
ROC_AUC =  0.978 ± 0.009
None
Sorted Sequence, Predictions, and Probabilities:
[['VHH' 1 1.0 0.9995585189805404]
 ['PP' 0 1.0 0.9602727517526459]
 ['HLPLP' 0 1.0 0.939002921786653]
 ['LLPHHADADY' 1 1.0 0.9294292670374633]
 ['VNPHDHQN' 1 1.0 0.9070931733294714]
 ['VIPAGYP' 1 1.0 0.9037433363871116]
 ['LWL' 0 1.0 0.8918619051773451]
 ['VHHANEN' 1 1.0 0.8629318351874767]
 ['LVNPHDHQN' 1 1.0 0.8523320553198657]
 ['AHK' 1 1.0 0.8022962853379847]
 ['LQSGDALRVPSGTTYY' 1 1.0 0.6466966399795839]
 ['VLPIPQ' 0 0.0 0.41879103567563636]
 ['SVPQPK' 0 0.0 0.3780097513661172]
 ['PLAQPA' 0 0.0 0.35163169881481193]
 ['EVPKA' 0 0.0 0.17302423872366363]
 ['IVF' 0 0.0 0.13816690196809345]
 